In [4]:
# importing libraries
%matplotlib inline
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import time

In [9]:

# reading users file:
u_cols = ['user_id', 'age', 'sex', 'occupation', 'zip_code']
users = pd.read_csv('u.user', sep='|', names=u_cols,encoding='latin-1')

# reading ratings file:
r_cols = ['user_id', 'movie_id', 'rating', 'unix_timestamp']
ratings = pd.read_csv('u.data', sep='\t', names=r_cols,encoding='latin-1')

# reading items file:
i_cols = ['movie id', 'movie title' ,'release date','video release date', 'IMDb URL', 'unknown', 'Action', 'Adventure',
'Animation', 'Children\'s', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy',
'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']
items = pd.read_csv('u.item', sep='|', names=i_cols,
encoding='latin-1')

# After loading the dataset, we should look at the content of each file (users, ratings, items).

# Looking at the user file
print("\nUser Data :")
print("shape : ", users.shape)
#print(users.head())

# We have 943 users in the dataset and each user has 5 features, i.e. user_ID, age, sex, occupation and zip_code. Now let’s look at the ratings file.

# Ratings Data
print("\nRatings Data :")
print("shape : ", ratings.shape)
#print(ratings.head())

# We have 100k ratings for different user and movie combinations. Now finally examine the items file.

# Item Data
print("\nItem Data :")
print("shape : ", items.shape)
#print(items.head())


User Data :
shape :  (943, 5)

Ratings Data :
shape :  (100000, 4)

Item Data :
shape :  (1682, 24)


In [ ]:
users.head()

,user_id,age,sex,occupation,zip_code
0,1,24,M,technician,85711
1,2,53,F,other,94043
2,3,23,M,writer,32067
3,4,24,M,technician,43537
4,5,33,F,other,15213


In [ ]:
ratings.head()

,user_id,movie_id,rating,unix_timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [ ]:
#converting the unix timestamp of ratings to a  date
from datetime import datetime
def time_stamp(k):
  return datetime.fromtimestamp(k).strftime('%d-%m-%Y')
ratings['rating_date'] = ratings['unix_timestamp'].apply(time_stamp)
ratings.head()

In [11]:
print(pd.DatetimeIndex(ratings['rating_date']).year.min())
print(pd.DatetimeIndex(ratings['rating_date']).year.max())

1997
1998


In [ ]:
#finding the no of days since the rating is given on movies from a specified date('01-11-1998')
date_format = "%d-%m-%Y"
def sub_dates(a):
  return (datetime.strptime('01-11-1998', date_format)-a).days   
def dat_strp(a):
    return datetime.strptime(a, date_format)
ratings['new_date']=ratings['rating_date'].apply(dat_strp)    
ratings['days_diff']=ratings['new_date'].apply(sub_dates) 
#conversion of no of days to years
ratings['years_diff']=round(ratings['days_diff']/365,2)
ratings.head() 

In [13]:
#dropping the below columns 
ratings.drop(['unix_timestamp','new_date'],axis=1,inplace=True)
ratings.head()

,user_id,movie_id,rating,rating_date,days_diff,years_diff
0,196,242,3,04-12-1997,332,0.91
1,186,302,3,04-04-1998,211,0.58
2,22,377,1,07-11-1997,359,0.98
3,244,51,2,27-11-1997,339,0.93
4,166,346,1,02-02-1998,272,0.75


In [ ]:
ratings['years_diff'].describe()

count    100000.000000
mean          0.837096
std           0.169676
min           0.530000
25%           0.690000
50%           0.860000
75%           0.970000
max           1.120000
Name: years_diff, dtype: float64

In [ ]:
#for these values select m=0.425 and n=0.25
m,n=0.425,0.25
ratings['weighted_diff']=m*ratings['years_diff']+n
ratings.head()

In [15]:
#Now I am gonna create new ratings which are time based/temporal
ratings['final_ratings']=round(ratings['rating']/ratings['weighted_diff'],2)
ratings.head()

,user_id,movie_id,rating,rating_date,days_diff,years_diff,weighted_diff,final_ratings
0,196,242,3,04-12-1997,332,0.91,0.63675,4.71
1,186,302,3,04-04-1998,211,0.58,0.49650,6.04
2,22,377,1,07-11-1997,359,0.98,0.66650,1.50
3,244,51,2,27-11-1997,339,0.93,0.64525,3.10
4,166,346,1,02-02-1998,272,0.75,0.56875,1.76


In [16]:
ratings['final_ratings'].describe()

count    100000.000000
mean          5.911761
std           2.037374
min           1.380000
25%           4.560000
50%           6.000000
75%           7.450000
max          10.520000
Name: final_ratings, dtype: float64

In [17]:
n_users = ratings.user_id.unique().shape[0]
n_items = ratings.movie_id.unique().shape[0]
data_matrix = np.zeros((n_users, n_items))

In [18]:
#basically here I am filling the respective user ids(line[1]) and movies(line[2]) with the value final ratings(line[8])
for line in ratings.itertuples():
    #print(line[1],line[2])
    data_matrix[line[1]-1, line[2]-1] = line[8]
data_matrix    

array([[6.93, 4.31, 5.96, ..., 0.  , 0.  , 0.  ],
       [7.42, 0.  , 0.  , ..., 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.  , ..., 0.  , 0.  , 0.  ],
       ...,
       [6.93, 0.  , 0.  , ..., 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.  , ..., 0.  , 0.  , 0.  ],
       [0.  , 9.35, 0.  , ..., 0.  , 0.  , 0.  ]])

In [20]:
#gonna use cosine similarity to determine similarities between movies and between users
from sklearn.metrics.pairwise import pairwise_distances 
user_similarity = pairwise_distances(data_matrix, metric='cosine') #similarities between users 
item_similarity = pairwise_distances(data_matrix.T, metric='cosine') #similarities between movies


In [21]:
def predict(ratings, similarity, type='user'):
    if type == 'user':
        mean_user_rating = ratings.mean(axis=1)
        #np.newaxis is used to make dimensions similar for ratings and mean_user_rating
        ratings_diff = (ratings - mean_user_rating[:, np.newaxis])
        #prediction for ratings on the basis of similarity
        pred = mean_user_rating[:, np.newaxis] + similarity.dot(ratings_diff) / np.array([np.abs(similarity).sum(axis=1)]).T
    elif type == 'item':
        pred = ratings.dot(similarity) / np.array([np.abs(similarity).sum(axis=1)])
    return pred

In [22]:
user_prediction = predict(data_matrix, user_similarity, type='user') #prediction of ratings using user-user similarity
item_prediction = predict(data_matrix, item_similarity, type='item') #prediction of ratings using item-item similarity

In [47]:
user_prediction[0]

array([3.29829425, 1.10668241, 0.92190528, ..., 0.53257039, 0.53145376,
       0.53049004])